In [ ]:
# ALPHA VERSION 10.0 ==========================================================
# Apply market indicators and basic commonality analysis
# to detect bottom and trigger the buying time via Telegram & Socket
# also perform back-testing record -> Save to Picture
# =============================================================================
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET & Socket requests
import requests
import urllib.parse
import json
import socket
# Library for Data Analysis
import pandas as pd

# Pre-define and CORE Function ================================================
# Define the list of 40 interesting coins
tuple_coin = (
  "UNIVNDC","CETUSVNDC","DOGEVNDC","JASMYVNDC","SOLVNDC",
  "MCAKEVNDC","SXPVNDC","CKBVNDC","EGPT1000VNDC","UXLINKVNDC",
  "MIAVNDC","MOODENGVNDC","EDU3VNDC","MEON1000VNDC","DOGSVNDC",
  "SUIVNDC","FOXYVNDC","BIGTIMEVNDC","TRUVNDC","OPVNDC",
  "LINKVNDC","XRPVNDC","1000CATVNDC","WIFVNDC","NEIROVNDC",
  "XLMVNDC","KSMVNDC","DOTVNDC","ENSVNDC","SANDVNDC",
  "GRASSVNDC","LYNXVNDC","AVAXVNDC","LISTAVNDC","TIAVNDC",
  "PEPE1000VNDC","WLDVNDC", "ADAVNDC", "HBARVNDC", "LDOVNDC"
)

# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Store last trigger timestamp, delay to avoid spam by continuous triggering
last_trigger = [0.0]*45
# Store the top 5 failure timestamp
last_failure = []
# Set the interval for the status message (in seconds)
interval = 60*60  # 1 hours in seconds
# Check point of important handling - in UNIX timestamp
check_point = time.time()

# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22, 0),(23, 59)),  # 10 PM to midnight
    ((0, 0),(6, 0))      # Midnight to 6 AM
]

# Function send to Telegram ===================================================
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"
    # Flag to sent --> Will send if flag = 0 afterall
    flag_send = 0
    current_time = datetime.now(timezone)

    for start, end in quiet_hours:
        # print(start[0], end[0])
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        quiet_start = current_time.replace(hour=start[0],
                                           minute=start[1],
                                           second=0,
                                           microsecond=0)
        quiet_end = current_time.replace(hour=end[0],
                                         minute=end[1],
                                         second=59,
                                         microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            flag_send += 1

    current_retry = 0
    while (current_retry <= max_retry) and (flag_send == 0):
        if current_retry > 0:
            print(f"Send Telegram: Retry #{current_retry}....................")
        try:
            # URL encode the message string
            encoded_message = urllib.parse.quote(message_string)
            # Construct the URL
            url1 = f"https://api.telegram.org/bot{TOKEN}/sendMessage?"
            url2 = f"chat_id={ID}&text={encoded_message}"
            # Send the request
            response = requests.get(url1 + url2, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Telegram: {e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "SEND TELEGRAM Max retries reached.")
                return False

# Function avoid Telegram spam if repeated trigger ============================
def check_trigger(message_string):
    global last_failure  # Declare last_failure as global

    current_time = time.time()
    # Remove timestamps older than 300 seconds
    last_failure = [timestamp for timestamp in last_failure
                    if current_time - timestamp <= 300]
    # Add the current timestamp
    last_failure.append(current_time)
    # Check if there are more than 5 errors within 300 seconds
    if len(last_failure) > 5:
        send_telegram(message_string)
        # Reset the list after sending the message
        last_failure = []

# Function get price 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 12h, 1d ============
def get_data(coin_name, time_unit, time_range, max_retry=5):
    url = "https://api-pro.goonus.io/perpetual/v1/klines"
    # Get the current Unix timestamp value in ms
    current_time = int(time.time()) * 1000
    # Define the parameters for the API request
    params = {
        "symbol": coin_name,
        "interval": time_unit,
        "endTime": current_time,
        # "limit": "600"
    }
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"GET Coin {coin_name}: Retry #{current_retry}........................")
        try:
            # Try request & get the Response during 5 seconds
            response = requests.get(url, params=params, timeout=5)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            historical_data = response.json()
            # Map the time_range matching with the total record
            while len(historical_data) < time_range:
                time_range -= 10
            # Check if the data has sufficient 100 records
            if len(historical_data) >= 100:
                # Take the latest "time_range" elements in the list
                historical_data = historical_data[-time_range:]
                # [Time,Open,High,Low,Close,Base qty,Quote qty,Time]
                # Filter to only include 'timestamp' and 'close'
                filtered_data = [[
                    # Convert Unix timestamp -> GMT +7 hours (+25200)
                    datetime.fromtimestamp(int(item[0])/1000, tz=pytz.utc)
                    .astimezone(timezone).strftime('%Y-%m-%d %H:%M:%S'),
                    # Close price
                    float(item[4]),
                    # Percentage difference
                    round(100*(float(item[2])-float(item[3]))/float(item[1]),2),
                    float(item[2]), float(item[3])
                ] for item in historical_data]
                # [timestamp, close, %price_diff, Highest, Lowest]
                # print(filtered_data)
                return filtered_data
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Not enough data for {coin_name} - only {len(historical_data)}"
                print(date_time, message)
                check_trigger(message)
                return None
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Request {coin_name}: (retry #{current_retry}): {e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: Max retries reached. Raising exception.")
                send_telegram(f"ERROR GET Coin {coin_name} (Limit Retried): {e}")
                return None

# Function call API to create in mySQL database ===============================
def create_database(coin_name, test_id, date_trigger,
                    trigger_value, trigger_note, max_retry=2):
    # Define the API endpoint and parameters
    url = "https://petiteo.com/api/create_backtest.php"
    params = {
        "coin_name": coin_name,
        "test_id": test_id,
        "date_trigger": date_trigger,
        "trigger_value": trigger_value,
        "trigger_note": trigger_note
    }
    current_retry = 0
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"CREATE DB Retry #{current_retry}........................")
        try:
            # Make the GET request
            response = requests.get(url, params=params, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            # Parse the JSON response
            response_data = response.json()
            if response_data.get("status") == "success":
                print(response_data.get("message"))
                return True
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Error CREATE: {response_data.get('message')}"
                print(date_time, message)
                send_telegram(message)
                return False
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{date_time}: CREATE DB failed\n{e}")
            send_telegram(f"CREATE DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: CREATE DB Max retries reached.")
                send_telegram(f"CREATE DB: Max retries reached.")
                return False

# Function Send Order via socket ==============================================
def trigger_socket(coin_name, order_price, variance):
    # Make JSON data payload
    data_send = {
        "coin_name": coin_name,
        "order_price": order_price,
        "variance": variance
    }
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(('localhost', 65432))
            s.sendall(json.dumps(data_send).encode('utf-8'))
    except ConnectionRefusedError:
        print("Connection refused. Ensure the server is running and listening on the correct port.")

# MARKET INDICATOR Function ===================================================
# Function to calculate the Simple Moving Average (SMA)
def calculate_sma(data, window=5):
    if len(data) < window:
        print("Not enough data to calculate SMA")
        check_trigger("Not enough data to calculate SMA")
        return pd.Series([None] * len(data))
    return data['close'].rolling(window=window).mean()

# Function to calculate the Relative Strength Index (RSI)
def calculate_rsi(data, window=15):
    if len(data) < window:
        print("Not enough data to calculate RSI")
        check_trigger("Not enough data to calculate RSI")
        return pd.Series([None] * len(data))

    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# MAIN PROGRAM ================================================================
if __name__ == '__main__':
    # Send log to Telegram
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time}: Alpha started"
    print(message)
    send_telegram(message)

    # Infinite loop to execute
    while True:
        # Example for fetching historical data for one-by-one coin
        for i in range(40):
            # Collect the short-term coin data (1 min)
            datacoin_short = get_data(tuple_coin[i], "1m", 600)
            if datacoin_short is None:
                continue

            # Convert to DataFrame for further process
            df = pd.DataFrame(datacoin_short,
                columns=['timestamp', 'close', 'percent', 'high', 'low'])
            # Format correct data type
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df['percent'] = df['percent'].astype(float)
            df['close'] = df['close'].astype(float)
            df['high'] = df['high'].astype(float)
            df['low'] = df['low'].astype(float)
            # Calculate average number of 'percent' last 30 records
            df['variance'] = df['percent'].rolling(window=30).mean()

            # Measure the Statistical Indicators =============================
            df['SMA_5'] = calculate_sma(df, 6)
            # Calculate the percentage difference
            # df['SMA_5_diff'] = df['SMA_5'].diff()
            df['SMA_5_diff'] = df['SMA_5'].diff() / df['SMA_5'].shift(1) * 100
            df['SMA_25'] = calculate_sma(df, 25)
            df['RSI_15'] = calculate_rsi(df)
            df['diff'] = 100 * (df['SMA_5'] - df['SMA_25']) / df['close']
            # Set the timestamp as the index
            # df.set_index('timestamp', inplace=True)
            # display(df)

            # Implement the swing strategy: buy the lowest
            average_before = df['SMA_5_diff'].iloc[-8:-3].mean()
            average_after = df['SMA_5_diff'].tail(3).mean()
            multiple = average_after * average_before

            # ALPHA Condition =================================================
            if  (df.iloc[-1]['diff'] <= -0.3) and\
                (df.iloc[-1]['RSI_15'] < 30) and\
                ((time.time() - last_trigger[i]) >= 900) and\
                (df.iloc[-1]['variance'] > 0.1) and\
                (average_before < 0) and (multiple < 0):
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"""	SMA5: {round(df.iloc[-1]['diff'], 3)}
	SMA25: {round(df.iloc[-1]['SMA_25'], 3)}
	Diff: {round(df.iloc[-1]['diff'], 3)}
	RSI15: {round(df.iloc[-1]['RSI_15'], 1)}
	Variance: {round(df.iloc[-1]['variance'], 3)}
	Before: {round(average_before, 3)}
	After: {round(average_after, 3)}"""
                print(date_time,tuple_coin[i]+"\n"+ message)
                send_telegram(date_time + ": " + tuple_coin[i] + "\n" + message)
                last_trigger[i] = time.time()
                # Create new Record in Back Testing database
                create_database(tuple_coin[i],
                                "BT07",
                                date_time,
                                df.iloc[-1]['close'].astype(str),
                                message)
                # Trigger signal via socket to place order
                trigger_socket(tuple_coin[i], df.iloc[-1]['close'],df.iloc[-1]['variance'])

        current_time = time.time()
        if (current_time - check_point) >= interval:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            check_point = current_time
            # Process Back Testing
            send_telegram("Alpha still working")

        time.sleep(5)